In [34]:
import sys
sys.path.insert(1, '../') 
from utils.loader import Loader
import tensorflow as tf
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, Flatten, Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [35]:
train = Loader.load_train()

In [ ]:
descriptions = [col for col in train.columns if "description" in col]

print(descriptions)

def custom_concat(row, cols):
    # Construir la descripción con lógica condicional basada en el valor de la celda
    parts = []
    for col_name in cols:  # Cambio para iterar solo sobre las columnas especificadas
        if col_name in row.index:  # Verificar que el nombre de la columna esté en el DataFrame
            value = row[col_name]
            if value == "no tiene" or not isinstance(value, str):
                parts.append(f"no tiene {col_name}")
            else:
                parts.append(str(value))  # Convertir a string para evitar problemas al unir
    # Unir todas las partes con espacios
    return ' '.join(parts)

# Aplicar la función al DataFrame
train['full_description'] = train.apply(custom_concat, axis=1, args=(descriptions,))
#train[descriptions].apply(lambda col: col.apply(lambda x: type(x).__name__).value_counts())

#descriptions.append("idx")
#descriptions.append("price")
#descriptions.append("km")

In [ ]:
filtered_columns = ["idx", "price", "km", "fuelType", "full_description"]
train = train[filtered_columns]
train

In [38]:
def preprocess_text(text):
    # Convertir a minúsculas y eliminar caracteres especiales (pero mantener los números)
    text = text.lower()
    text = re.sub(r'[^\w\s\d]', '', text)  # Eliminar signos de puntuación, pero mantener números
    tokens = text.split()
    return tokens

train['tokens'] = train['full_description'].apply(preprocess_text)
model_w2v = Word2Vec(sentences=train['tokens'], vector_size=100, window=1, min_count=3, workers=8)
word_vectors = model_w2v.wv

# Crear una función para convertir cada descripción a un embedding promedio
def get_average_embedding(tokens, model):
    embeddings = [model[word] for word in tokens if word in model]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)  # Si no hay palabras en el modelo, devolver vector de ceros

# Aplicar el embedding a todas las descripciones
train['embedding'] = train['tokens'].apply(lambda x: get_average_embedding(x, word_vectors))

In [39]:
# Asumiendo que train es tu DataFrame y 'price' es la columna objetivo
X_km = train[['km']].values  # Características numéricas
y = train['price'].values  # Etiqueta objetivo

# Dividir el conjunto de datos en entrenamiento y prueba
X_train_km, X_test_km, y_train, y_test = train_test_split(X_km, y, test_size=0.2, random_state=42)

In [ ]:
# Asumiendo que ya has creado los embeddings
X_textual = np.vstack(train['embedding'].values)

# Dividir los embeddings textuales en entrenamiento y prueba
X_train_text, X_test_text = train_test_split(X_textual, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_km_scaled = scaler.fit_transform(X_train_km)
X_test_km_scaled = scaler.transform(X_test_km)

In [ ]:
# Crear entradas separadas
input_km = Input(shape=(1,), name='input_km')
input_text = Input(shape=(100,), name='input_text')  # Asegúrate de que 100 coincida con la dimensión de tus embeddings

# Combinar las entradas
combined_input = concatenate([input_km, input_text])

# Añadir capas densas después de la combinación
x = Dense(128, activation='relu')(combined_input)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)  # Capa de salida para regresión

# Crear el modelo
model = Model(inputs=[input_km, input_text], outputs=output)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(0.01), loss='huber', metrics=['mae'])

# Resumen del modelo
model.summary()

In [ ]:
history = model.fit(
    [X_train_km_scaled, X_train_text], y_train,
    epochs=10, batch_size=32,
    validation_data=([X_test_km_scaled, X_test_text], y_test)
)

In [ ]:
plt.figure(figsize=(10, 6))

# Plotea la pérdida del conjunto de entrenamiento
plt.plot(history.history['loss'], label='Training Loss')

# Plotea la pérdida del conjunto de validación
plt.plot(history.history['val_loss'], label='Validation Loss')

# Añade título y etiquetas
plt.title('Model Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss (Mean Squared Error)')

# Añade leyenda
plt.legend()

# Añade cuadrícula
plt.grid(True)

# Muestra el gráfico
plt.show()